Cell 1 — Setup & config (edit DATA_PATH / SQA_PARAMS here)

In [36]:
# Cell 1: Setup & configuration
# Edit DATA_PATH to point to problem_data.json, problem_data_8x_seq.json, or problem_data_12x.json as desired.
DATA_PATH = "problem_data_12x.json"
OUT_ASSIGN_JSON = "sqa_assignment_final.json"
OUT_ASSIGN_REPAIRED = "sqa_assignment_final_repaired.json"

# SQA params - tune to your compute budget
SQA_PARAMS = {
    "num_reads": 256,        # how many reads per restart
    "num_sweeps": 15000,     # sweeps per read (increase for harder mixing)
    "trotter_slices": 16,
    "beta_start": 0.1,
    "beta_end": 5.0,
    "beta_schedule_type": "geometric",
    "num_restarts": 6,
    "seed": 12345
}

# QUBO scaling target (keep coefficients modest for SQA)
TARGET_MAX_COEF = 3.0

# Penalty / tuning defaults (will be read from JSON if present)
DEFAULT_TUNING = {"gamma": 20.0, "eta": 50.0, "tau": 15.0, "varphi": 15.0, "psi": 0.3}

# Required packages:
# pip install dimod dwave-neal


Cell 2 — Imports + load JSON + quick diagnostics

In [37]:
# Cell 2: imports and load data
import json, math, random, time
from collections import defaultdict
from pprint import pprint

import dimod
from neal import SimulatedAnnealingSampler

with open(DATA_PATH, "r") as f:
    data = json.load(f)

# Extract structures
flights_list = data["flights"]
tails_list = data["tails"]
costs = data["costs"]
max_costs = data["max_costs"]
impossible_pairings = set(tuple(p) for p in data.get("impossible_pairings", []))
tuning_params = data.get("tuning_parameters", {})

flights = {f["id"]: f for f in flights_list}
tails = {t["id"]: t for t in tails_list}

print("Loaded dataset:", DATA_PATH)
print("Flights:", len(flights), "Tails:", len(tails), "Impossible pairs:", len(impossible_pairings))
# simple check: min/max feasible tails per flight
valid_counts = []
for f_id, f in flights.items():
    valid = [t_id for t_id,t in tails.items()
             if t["fleet"]==f["fleet_req"] and t["capacity"]>=f["seats_req"] and f_id in costs and t_id in costs[f_id]]
    valid_counts.append(len(valid))
print("Feasible tails per flight: min", min(valid_counts), "max", max(valid_counts))


Loaded dataset: problem_data_12x.json
Flights: 360 Tails: 17 Impossible pairs: 1812
Feasible tails per flight: min 1 max 7


Cell 3 — Build QUBO (only feasible pairs) — HA, HB1, HB3, HC and returns Q dict

In [38]:
# Cell 3: Build QUBO (filtering invalid pairs)
from collections import defaultdict

def var_name(f_id, t_id):
    return f"{f_id}|{t_id}"

def parse_var_name(s):
    if isinstance(s, str) and "|" in s:
        return tuple(s.split("|",1))
    return None

def build_qubo_from_data(data, tuning_override=None):
    # unpack
    # Build QUBO with higher eta penalty


    flights = {f["id"]: f for f in data["flights"]}
    tails = {t["id"]: t for t in data["tails"]}
    costs = data["costs"]
    max_costs = data["max_costs"]
    tp = data.get("tuning_parameters", {})
    # base weights
    gamma  = float(tp.get("gamma", DEFAULT_TUNING["gamma"]))
    eta    = float(tp.get("eta", DEFAULT_TUNING["eta"]))
    tau    = float(tp.get("tau", DEFAULT_TUNING["tau"]))
    varphi = float(tp.get("varphi", DEFAULT_TUNING["varphi"]))
    psi    = float(tp.get("psi", DEFAULT_TUNING["psi"]))
    if tuning_override:
        gamma  = tuning_override.get("gamma", gamma)
        eta    = tuning_override.get("eta", eta)
        tau    = tuning_override.get("tau", tau)
        varphi = tuning_override.get("varphi", varphi)
        psi    = tuning_override.get("psi", psi)

    # determine valid pairs
    valid_pairs = set()
    for f_id, f in flights.items():
        listed = set(costs.get(f_id, {}).keys())
        for t_id, t in tails.items():
            fleet_ok = (t["fleet"] == f["fleet_req"])
            seats_ok = (t["capacity"] >= f["seats_req"])
            if fleet_ok and seats_ok and (t_id in listed):
                valid_pairs.add((f_id, t_id))

    Q = defaultdict(float)
    def add_lin(v,w): Q[(v,v)] += w
    def add_quad(u,v,w):
        if u==v:
            Q[(u,u)] += w
        else:
            a,b = (u,v) if u < v else (v,u)
            Q[(a,b)] += w

    # HA: one-hot per flight over only valid tails
    for f_id in flights.keys():
        tail_block = [var_name(f_id,t) for (ff,t) in valid_pairs if ff==f_id]
        if not tail_block:
            continue
        for v in tail_block:
            add_lin(v, -gamma)            # linear part -gamma
        for i in range(len(tail_block)):
            for j in range(i+1, len(tail_block)):
                add_quad(tail_block[i], tail_block[j], 2.0 * gamma)

    # HB1: impossible pairs (only if both (f,t) are valid)
    for (f1, f2) in data.get("impossible_pairings", []):
        for t_id in tails.keys():
            if (f1, t_id) in valid_pairs and (f2, t_id) in valid_pairs:
                add_quad(var_name(f1,t_id), var_name(f2,t_id), eta)

    # HB3: maintenance linear penalty if tail has maintenance day (applies to valid pairs)
    for t_id, t in tails.items():
        maint = t.get("maintenance", None)
        if not maint:
            continue
        mday = maint.get("day")
        for f_id, f in flights.items():
            if f["departure"]["day"] == mday and (f_id, t_id) in valid_pairs:
                add_lin(var_name(f_id, t_id), tau)

    # HC: normalized cost linear term (only for valid pairs)
    for f_id, tails_cost in costs.items():
        mc = max_costs.get(f_id, 1.0)
        for t_id, c in tails_cost.items():
            if (f_id, t_id) in valid_pairs:
                add_lin(var_name(f_id, t_id), psi * (c / mc))

    mapping = {var_name(f,t): (f,t) for (f,t) in sorted(valid_pairs)}
    return dict(Q), mapping, valid_pairs, {"gamma":gamma,"eta":eta,"tau":tau,"varphi":varphi,"psi":psi}

# Build
Q_dict, mapping, valid_pairs, used_tuning = build_qubo_from_data(data)
print("Built QUBO: variables (valid pairs):", len(mapping))
print("Tuning used:", used_tuning)


Built QUBO: variables (valid pairs): 1656
Tuning used: {'gamma': 20.0, 'eta': 50.0, 'tau': 15.0, 'varphi': 15.0, 'psi': 0.3}


Cell 4 — Rescale QUBO, convert to BQM

In [39]:
# Cell 4: Rescale and convert to dimod BQM
def rescale_qubo(Q_dict, target_max=TARGET_MAX_COEF):
    if not Q_dict: return Q_dict, 1.0
    m = max(abs(v) for v in Q_dict.values())
    if m <= 0: return dict(Q_dict), 1.0
    scale = (m / target_max) if m > target_max else 1.0
    Q_rescaled = {k: v/scale for k,v in Q_dict.items()} if scale != 1.0 else dict(Q_dict)
    return Q_rescaled, scale

def qubo_to_bqm(Q_dict):
    linear = {}
    quadratic = {}
    for (u,v), w in Q_dict.items():
        if u==v:
            linear[u] = linear.get(u, 0.0) + w
        else:
            a,b = (u,v) if u < v else (v,u)
            quadratic[(a,b)] = quadratic.get((a,b), 0.0) + w
    return dimod.BinaryQuadraticModel(linear, quadratic, 0.0, vartype=dimod.BINARY)

Q_rescaled, scale = rescale_qubo(Q_dict, TARGET_MAX_COEF)
bqm = qubo_to_bqm(Q_rescaled)
print("Rescale factor:", scale, "BQM vars:", len(bqm.variables), "quadratic:", len(bqm.quadratic))


Rescale factor: 16.666666666666668 BQM vars: 1656 quadratic: 5880


Cell 5 — Run SQA with restarts (neal)

In [40]:
# Cell 5: Run SQA (neal) with restarts
sampler = SimulatedAnnealingSampler()
rng = random.Random(SQA_PARAMS.get("seed", 0))

def run_sqa_restarts(bqm, params):
    best_e = float("inf")
    best_samp = None
    all_ss = []
    for r in range(params.get("num_restarts", 4)):
        seed = rng.randint(1, 2**31-2)
        ss = sampler.sample(
            bqm,
            num_reads=params.get("num_reads", 128),
            num_sweeps=params.get("num_sweeps", 10000),
            num_trotter_slices=params.get("trotter_slices", 16),
            beta_range=(params.get("beta_start",0.1), params.get("beta_end",5.0)),
            beta_schedule_type=params.get("beta_schedule_type","geometric"),
            seed=seed
        )
        all_ss.append(ss)
        if ss.first.energy < best_e:
            best_e = ss.first.energy
            best_samp = ss.first.sample
    try:
        combined = dimod.concatenate(all_ss)
    except Exception:
        combined = all_ss[0]
    return best_samp, best_e, combined

print("Starting SQA... (this may take time depending on params)")
t0 = time.time()
best_sample, best_energy, combined = run_sqa_restarts(bqm, SQA_PARAMS)
t_elapsed = time.time() - t0
print("SQA done. Best energy (rescaled):", best_energy, "Time(s):", round(t_elapsed,1))
# Convert sample keys to strings (var names) and ints
best_sample = {str(k): int(v) for k,v in best_sample.items()}


Starting SQA... (this may take time depending on params)
SQA done. Best energy (rescaled): -424.96988534679394 Time(s): 1311.9


Cell 6 — Decode sample → assignment + basic greedy repair (one-hot) and validate

In [41]:
# Cell 6: Decode + greedy one-hot repair and initial validation
# chosen ones per flight from sampler
chosen = defaultdict(list)
for var, val in best_sample.items():
    if val != 1: continue
    parsed = parse_var_name(var)
    if parsed is None: continue
    f_id, t_id = parsed
    if (f_id, t_id) in valid_pairs:
        chosen[f_id].append(t_id)

# Greedy repair: ensure exactly one tail per flight (choose cheapest among selected or cheapest valid)
assign = {}
for f_id in flights.keys():
    cands = chosen.get(f_id, [])
    if len(cands) == 1:
        assign[f_id] = cands[0]
    elif len(cands) > 1:
        assign[f_id] = min(cands, key=lambda t: costs.get(f_id, {}).get(t, float('inf')))
    else:
        # pick cheapest valid tail overall
        valid_for_f = [t for (ff,t) in valid_pairs if ff==f_id]
        if valid_for_f:
            assign[f_id] = min(valid_for_f, key=lambda t: costs.get(f_id, {}).get(t, float('inf')))
        else:
            assign[f_id] = None

# Quick validation
def validate_assignment(mapping):
    viol_fc = []
    for f,t in mapping.items():
        if t is None:
            viol_fc.append((f,t,"unassigned"))
            continue
        if t not in tails:
            viol_fc.append((f,t,"unknown_tail"))
            continue
        fobj = flights[f]; tobj = tails[t]
        if tobj["fleet"] != fobj["fleet_req"]:
            viol_fc.append((f,t,"fleet_mismatch"))
        if tobj["capacity"] < fobj["seats_req"]:
            viol_fc.append((f,t,"insufficient_capacity"))
    # impossible pairs
    tail_to_f = defaultdict(list)
    for f,t in mapping.items():
        if t is not None:
            tail_to_f[t].append(f)
    imp_viol = []
    impset = set(tuple(p) for p in data.get("impossible_pairings", []))
    for t, flist in tail_to_f.items():
        fs = sorted(flist)
        for i in range(len(fs)):
            for j in range(i+1, len(fs)):
                a,b = fs[i], fs[j]
                if (a,b) in impset or (b,a) in impset:
                    imp_viol.append((t,a,b))
    return viol_fc, imp_viol

viol_fc, imp_viol = validate_assignment(assign)
print("Assigned flights:", sum(1 for v in assign.values() if v is not None), "/", len(flights))
print("Fleet/capacity violations:", len(viol_fc))
print("Impossible-pair violations:", len(imp_viol))
if len(viol_fc): pprint(viol_fc[:10])
if len(imp_viol): pprint(imp_viol[:10])

# Save preliminary assignment
with open(OUT_ASSIGN_JSON, "w") as fo:
    json.dump(assign, fo, indent=2)
print("Saved preliminary SQA assignment to:", OUT_ASSIGN_JSON)


Assigned flights: 360 / 360
Fleet/capacity violations: 0
Impossible-pair violations: 0
Saved preliminary SQA assignment to: sqa_assignment_final.json


Cell 7 — Robust iterative repair to remove impossible-pair violations (guaranteed if feasible alternatives exist)

In [42]:
# Cell 7: Robust repair for impossible-pair violations (deterministic)
# Loads the preliminary assignment `assign` from Cell 6 and fixes impossible pair conflicts
valid_tails_for_f = {}
for f_id, f in flights.items():
    valid_tails_for_f[f_id] = [t for t in costs.get(f_id, {})
                               if tails[t]['fleet']==f['fleet_req'] and tails[t]['capacity']>=f['seats_req']]

def find_imp_violations(mapping):
    tail_to_f = defaultdict(list)
    for f,t in mapping.items():
        if t is not None:
            tail_to_f[t].append(f)
    viol = []
    for t, fl in tail_to_f.items():
        fs = sorted(fl)
        for i in range(len(fs)):
            for j in range(i+1, len(fs)):
                a,b = fs[i], fs[j]
                if (a,b) in impossible_pairings or (b,a) in impossible_pairings:
                    viol.append((t,a,b))
    return viol

# Build tail->flights map
tail_to_f = defaultdict(set)
for f,t in assign.items():
    if t is not None:
        tail_to_f[t].add(f)

viol = find_imp_violations(assign)
iteration = 0
max_iter = 20000
while viol and iteration < max_iter:
    iteration += 1
    t, f1, f2 = viol[0]  # pick first violation
    # try reassigning the one with cheaper sacrifice (min delta)
    best_change = None
    for f in (f1, f2):
        cur_t = assign[f]
        cur_cost = costs[f][cur_t]
        best_alt = None; best_delta = float('inf')
        for cand in valid_tails_for_f[f]:
            if cand == cur_t: continue
            # ensure assigning f->cand doesn't conflict with flights already on cand
            conflict = False
            for other in tail_to_f[cand]:
                if other == f: continue
                if (f, other) in impossible_pairings or (other, f) in impossible_pairings:
                    conflict = True; break
            if conflict: continue
            delta = costs[f][cand] - cur_cost
            if delta < best_delta:
                best_delta = delta; best_alt = cand
        if best_alt is not None:
            best_change = (best_delta, f, best_alt)
    if not best_change:
        # cannot resolve this violation right now; try next violation
        viol.pop(0)
        if not viol:
            viol = find_imp_violations(assign)
        continue
    # perform best change
    delta, f_sel, new_tail = best_change
    old_tail = assign[f_sel]
    tail_to_f[old_tail].remove(f_sel)
    assign[f_sel] = new_tail
    tail_to_f[new_tail].add(f_sel)
    # recompute violations
    viol = find_imp_violations(assign)

# Final validation
viol_fc2, imp_viol2 = validate_assignment(assign)
print("Repair iterations:", iteration)
print("Final fleet/capacity violations:", len(viol_fc2))
print("Final impossible-pair violations:", len(imp_viol2))
if imp_viol2:
    pprint(imp_viol2[:10])

# Save repaired assignment
with open(OUT_ASSIGN_REPAIRED, "w") as fo:
    json.dump(assign, fo, indent=2)
print("Saved repaired assignment to:", OUT_ASSIGN_REPAIRED)


Repair iterations: 0
Final fleet/capacity violations: 0
Final impossible-pair violations: 0
Saved repaired assignment to: sqa_assignment_final_repaired.json


Cell 8 — Final scoring and optional local-improvement (pairwise swap + block MILP)

In [44]:
# Cell 8: Final scoring and optional local improvements
def total_raw_cost(mapping):
    s = 0.0
    for f,t in mapping.items():
        if t is None: return float('inf')
        s += costs[f][t]
    return s

def total_normalized_cost(mapping):
    s = 0.0
    for f,t in mapping.items():
        if t is None: return float('inf')
        s += costs[f][t] / max_costs[f]
    return s

final_assign = assign  # repaired assignment from Cell 7
raw = total_raw_cost(final_assign)
norm = total_normalized_cost(final_assign)
print("Final assigned flights:", len([f for f in final_assign if final_assign[f] is not None]), "/", len(flights))
print("Final raw cost:", raw)
print("Final normalized cost:", norm)

# Optional: run lightweight pairwise-swap local-improvement (fast)
do_pairwise_swap = True
if do_pairwise_swap:
    import random, time
    from collections import defaultdict
    start = time.time()
    tail_map = defaultdict(set)
    for f,t in final_assign.items(): tail_map[t].add(f)
    impset = set(tuple(p) for p in data.get("impossible_pairings", []))
    best_assign = dict(final_assign)
    best_cost = raw

    # attempt many random pair swaps
    trials = 50000
    flies = list(best_assign.keys())
    for i in range(trials):
        a = random.choice(flies); b = random.choice(flies)
        if a==b: continue
        ta = best_assign[a]; tb = best_assign[b]
        if ta==tb: continue
        # quick feasibility checks for swap
        def can_put(flight, new_tail, current_assign, tail_map):
            if new_tail not in costs.get(flight, {}): return False
            tobj = tails[new_tail]; fobj = flights[flight]
            if tobj['fleet'] != fobj['fleet_req'] or tobj['capacity'] < fobj['seats_req']:
                return False
            for other in tail_map[new_tail]:
                if other==flight: continue
                if (flight, other) in impset or (other, flight) in impset:
                    return False
            return True
        if not (can_put(a,tb,best_assign,tail_map) and can_put(b,ta,best_assign,tail_map)):
            continue
        new_cost = best_cost - costs[a][ta] - costs[b][tb] + costs[a][tb] + costs[b][ta]
        if new_cost < best_cost - 1e-9:
            # accept swap
            best_assign[a], best_assign[b] = tb, ta
            tail_map[ta].remove(a); tail_map[tb].remove(b)
            tail_map[ta].add(b); tail_map[tb].add(a)
            best_cost = new_cost
    elapsed = time.time() - start
    print("Pairwise-swap trial done. Improved cost:", best_cost, "time(s):", round(elapsed,1))
    final_assign = best_assign

# Save final improved assignment
with open(OUT_ASSIGN_REPAIRED, "w") as fo:
    json.dump(final_assign, fo, indent=2)
print("Final assignment saved to:", OUT_ASSIGN_REPAIRED)
print("Final raw cost:", total_raw_cost(final_assign))
print("Final normalized cost:", total_normalized_cost(final_assign))

# Show sample of assignments
sample_keys = sorted(list(final_assign.keys()))[:40]
for k in sample_keys:
    print(k, "->", final_assign[k])


Final assigned flights: 360 / 360
Final raw cost: 48712.559999999954
Final normalized cost: 340.5619251709143
Pairwise-swap trial done. Improved cost: 48419.209999999985 time(s): 0.1
Final assignment saved to: sqa_assignment_final_repaired.json
Final raw cost: 48419.20999999998
Final normalized cost: 338.98947756489997
F1 -> A101
F10 -> Q171
F100 -> N141
F101 -> H808
F102 -> M131
F103 -> J101
F104 -> D404
F105 -> K111
F106 -> N141
F107 -> N141
F108 -> M131
F109 -> Q171
F11 -> H808
F110 -> I909
F111 -> Q171
F112 -> P161
F113 -> O151
F114 -> J101
F115 -> P161
F116 -> I909
F117 -> L121
F118 -> A101
F119 -> N141
F12 -> F606
F120 -> H808
F121 -> C303
F122 -> A101
F123 -> N141
F124 -> D404
F125 -> E505
F126 -> C303
F127 -> G707
F128 -> F606
F129 -> P161
F13 -> N141
F130 -> Q171
F131 -> H808
F132 -> C303
F133 -> Q171
F134 -> D404
